In diesem Tutorial lernen wir die Funktionsweise des **Connectionist Neuron** interaktiv kennen.

Um dieses Tutorial auszuführen brauchen wir interaktives Plotting. Dieses ist in Jupyter Notebooks prinzipiell einfach möglich mit der integrierten Bibliothek `ipywidgets`. Diese Bibliothek erfordert allerdings versionsabhängig verschiedene sogenannte Backends, nämlich entweder

```python
%matplotlib widget
# oder
%matplotlib notebook
```

die mit der sogenannten Jupyter-Magic (das Prozentzeichen) aktiviert werden müssen. Jedes Mal, wenn wir eines dieser Backends aktivieren, kann es nicht mehr geändert werden. Wollen wir uns Backend danach ändern, müssen wir das **Notebook neu starten**. Dies tun wir über *Kernel -> Restart* oben in der Taskleiste.

In [1]:
%matplotlib widget
# oder
# %matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 1. Entscheidungsgrenzen

Nun brauchen wir noch eine begleitende Datei, mit Code, den wir in dieses Notebook importieren wollen. Der Code selbst ist nicht relevant, wir wollen ihn nur verwenden.

In [2]:
from utils_logistic import get_interactive_logistic_regression_advanced

Als letztes laden wir einen Toy-Datensatz:

In [3]:
# Laden die Rohdaten - Pfad muss eventuell geändert werden

# Option - Numpy
# data = np.loadtxt("../data/ZonkoAndTydum.csv", delimiter=",", skiprows=1)
# Option - Pandas
data = pd.read_csv("../data/ZonkoAndTydum.csv")

# Teilen die Daten in Beobachtungen und Labels
X = data.iloc[:, 0:2].values
y = data.iloc[:, 2].values

In [4]:
data

,x1 [Durchmesser],x2 [Gewicht],y [Typ]
0,0.365,0.708,0
1,0.543,-0.268,0
2,-0.401,0.643,0
3,0.866,-0.796,0
4,-0.386,0.742,0
...,...,...,...
195,0.784,0.616,1
196,0.411,-0.005,1
197,0.930,0.225,1
198,1.288,0.184,1


In [5]:
interactive_plot, ui = get_interactive_logistic_regression_advanced(X, y)
display(interactive_plot, ui)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

## 2. Modell trainieren

In [6]:
# TODO: Laden des house_location Datensatzes
train_data = pd.read_csv("../data/house_location/house_location_data.csv")
test_data = pd.read_csv("../data/house_location/house_location_new_data_with_labels.csv")

X_train = train_data.loc[:, ["price_per_sqft", "elevation"]].values.astype(np.float64)
y_train = train_data.loc[:, "in_sf"].values

X_test = test_data.loc[:, ["price_per_sqft", "elevation"]].values.astype(np.float64)
y_test = test_data.loc[:, "in_sf"].values

# TODO: Skalieren des House Location Datensatzes
maximum_price_per_sqft = 1. * np.max(X_train[:, 0])
maximum_elevation = 1. * np.max(X_train[:, 1])

X_train_scaled = X_train.copy()
X_train_scaled[:, 0] = X_train[:, 0] / maximum_price_per_sqft
X_train_scaled[:, 1] = X_train[:, 1] / maximum_elevation

X_test_scaled = X_test.copy()
X_test_scaled[:, 0] = X_test[:, 0] / maximum_price_per_sqft
X_test_scaled[:, 1] = X_test[:, 1] / maximum_elevation

X_train_scaled

array([[0.21386655, 0.44117647],
       [0.32601608, 0.02941176],
       [0.16735492, 0.7605042 ],
       [0.1406216 , 0.18067227],
       [0.76983265, 0.09663866],
       [0.28645947, 0.21008403],
       [0.1884373 , 0.28991597],
       [0.5520539 , 0.01260504],
       [0.19626168, 0.13445378],
       [0.1573571 , 0.29831933],
       [0.16887633, 0.45378151],
       [0.13323191, 0.7394958 ],
       [0.2175614 , 0.19327731],
       [0.14018692, 0.75210084],
       [0.26994132, 0.04201681],
       [0.33427516, 0.03361345],
       [0.79700065, 0.08823529],
       [0.12149533, 0.20168067],
       [0.1790915 , 0.10084034],
       [0.20973701, 0.06722689],
       [0.25755271, 0.04201681],
       [0.1864812 , 0.04201681],
       [0.2486416 , 0.01260504],
       [0.18604651, 0.13865546],
       [0.24212128, 0.04621849],
       [0.4242556 , 0.04201681],
       [0.22886329, 0.43277311],
       [0.0695501 , 0.05042017],
       [0.28885025, 0.07983193],
       [0.30123886, 0.04201681],
       [0.

In [7]:
# Wir untersuchen die Trainings- und Testdaten qualitativ.
# Einmal mit Skalierung und einmal ohne

# ohne Skalierung
plt.figure(figsize=(8, 6))
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, alpha=0.5)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, marker="x")
plt.xlabel("price per sqft")
plt.ylabel("elevation");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# mit Skalierung
plt.figure(figsize=(8, 6))
plt.scatter(X_train_scaled[:, 0], X_train_scaled[:, 1], c=y_train, alpha=0.5)
plt.scatter(X_test_scaled[:, 0], X_test_scaled[:, 1], c=y_test, marker="x")
plt.xlabel("price per sqft / maximum price per sqft")
plt.ylabel("elevation / maximum elevation");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1. Interaktiv trainieren

Wir versuchen zunächst eine Intuition für das Modell zu entwickeln und benutzen dazu interaktives Plotting. Um eine ansprechende Visualisierung zur gewährleisten ist außerdem erstmals das "Skalieren" der Features relevant. Erst zu einem späteren Zeitpunkt werden wir dieses Skalieren genauer kennenlernen.

In [11]:
# TODO: Importieren des interaktiven Fitting-Tools
from utils_logistic import InteractiveConnectionistNeuron

In [12]:
# TODO: Trainieren
model = InteractiveConnectionistNeuron()
model.fit(X_train_scaled, y_train)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.1, description='w1', max=6.0, min=-6.0, step=0.05), FloatSlider(valu…

In [12]:
# die Parameter des Modells können ausgegeben werden
print(model.w1)
print(model.w2)
print(model.bias)

0.1
1.0
0.0


In [13]:
# TODO: Modell evaluieren

# Option 1 - naives Python mit for-Schleife
def predict(X, w1, w2, bias):
    
    n_rows = X.shape[0]
    
    y_pred = np.zeros(n_rows)
    
    for i in range(n_rows):
        x_i = X[i, :]

        h = x_i[0] * w1 + x_i[1] * w2 - bias
        
        if h >= 0:
            y_pred[i] = 1
        else:
            y_pred[i] = 0
        
    return y_pred


# Option 2 - Numpy mit np.dot
def predict(X, w1, w2, bias):
    
    n = len(X)
    y_pred = np.zeros(n)
    
    w = np.array([w1, w2])
    
    for i in range(n):
        x_i = X[i, :]
        
        # vorher: h = x_i[0] * w1 + x_i[1] * w2 - bias
        # jetzt:
        h = np.dot(x_i, w) - bias
        
        y_pred[i] = int(h >= 0)
        
    return y_pred


# Option 3 - Numpy mit np.matmul
def predict(X, w1, w2, bias):   
    w = np.array([w1, w2])
    y_pred = ((np.matmul(X, w) - bias) >= 0).astype(np.int64)
    return y_pred


# Eine Hilfsfunktion zur Berechnung der Genauigkeit
def accuracy_score(y_true, y_pred):
    return (y_true == y_pred).sum() / len(y_pred)


y_pred = predict(X_test, model.w1, model.w2, model.bias)

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.5454545454545454

In [14]:
# TODO: Vorhersagen auf unbekannten Datenpunkten
x_new = np.array([[3600, 125]])

x_new[:, 0] = x_new[:, 0] / maximum_price_per_sqft
x_new[:, 1] = x_new[:, 1] / maximum_elevation

predict(x_new, model.w1, model.w2, model.bias)

array([1])

### 2.2. Training mit Scikit-Learn

In diesem Abschnitt vergleichen wir das interaktive Training mit einem von Scikit-Learn bereitgestellten Modell.

In [15]:
# TODO: Trainieren mit der Scikit-Learn LogisticRegression
# Logistische Regression ~= Connectionist Neuron mit Transferfunktion als Sprungfunktion
from sklearn.linear_model import LogisticRegression

In [16]:
scikit_model = LogisticRegression()

In [17]:
# Das Attribut `coef_` ist zunächst nicht bekannt.
# Führen Sie den folgenden Code aus, um die Fehlermeldung zu überprüfen

# print(scikit_model.coef_)

In [18]:
# Nach dem Fitten des Modells sind die Attribute `coef_` und `intercept_` bekannt
scikit_model.fit(X_train_scaled, y_train)

# coef_ = weights
print(scikit_model.coef_)
# intercept_ = bias/theta
print(scikit_model.intercept_)

[[-4.57032638  5.13341801]]
[0.60800476]


In [19]:
# TODO: Modell evaluieren
y_pred_test = scikit_model.predict(X_test_scaled)

scikit_accuracy = accuracy_score(y_test, y_pred_test)
scikit_accuracy

0.7474747474747475

In [20]:
# TODO: Vorhersagen auf unbekannten Datenpunkten
scikit_model.predict(x_new)

array([1])